## Topic Modelling - Udacity

In [1]:
import sys

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pprint import pprint

# Plotting tools
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
path = '--'
%run ./Text_Normalization_Function.ipynb

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [3]:
df = pd.read_csv(path+"Udacity_txt.txt", delimiter = "\t", header = None)

In [4]:
# remove http, and @ and username
for index, row in df.iterrows():
    row[0] = str(row[0]).split(" ")
    for i in range(len(row[0])):
        row[0][i] = "".join([word for word in row[0][i].split()
                            if "http" not in word and "@" not in word and "<" not in word])
    row[0] = ",".join(row[0])
    row[0] = str(row[0]).replace(",", " ")

**** LDA Topic Model ****

In [5]:
normalized_udacity = normalize_corpus(df[1])
bow_vectorizer_udacity = CountVectorizer()
bow_udacity_corpus = bow_vectorizer_udacity.fit_transform(normalized_udacity)
bow_feature_names_udacity = bow_vectorizer_udacity.get_feature_names()
bow_table = pd.DataFrame(data = bow_udacity_corpus.todense(), columns = bow_feature_names_udacity)
bow_table.head()

,000s,07psyrntwf,08rnt87bvt,0biqvjgaqm,0h2plto95e,0rduy8mluy,0rrogcixwm,0sghq6jdqn,100daysofcode,100daysofmlcode,...,zicgwx6krw,zillow,zmqojpcpis,znhlyvqxgf,znsrym6qvm,zop6oyyidj,zpjediouqc,zqv1itixrm,zsgegskg0a,zvjekku9vy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# set up the model 
no_topics = 3
lda_udacity_corpus = LatentDirichletAllocation(n_components=no_topics, 
                                           doc_topic_prior = 0.3,
                                           topic_word_prior = 0.2).fit(bow_udacity_corpus)


C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [7]:
no_top_words = 10
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

topic_words = get_topic_words(vectorizer = bow_vectorizer_udacity, 
                              lda_model = lda_udacity_corpus, 
                              n_words = no_top_words)
pd.DataFrame(topic_words, 
             columns = ["word_" + str(i) for i in range(no_top_words)],
             index = ["Topic_" + str(i) for i in range(len(topic_words))]) 

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
Topic_0,learn,google,free,computer,job,science,want,web,today,build
Topic_1,notebook,idea,video,graffiti,live,top,great,sound,projectjupyter,annotation
Topic_2,nanodegree,data,day,learning,100daysofcode,course,program,new,machine,amazon


In [8]:
word_weights = lda_udacity_corpus.components_ / lda_udacity_corpus.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T, 
                               index = bow_feature_names_udacity, 
                               columns = ["Topic_" + str(i) for i in range(no_topics)])
word_weights_df.head(10)

,Topic_0,Topic_1,Topic_2
000s,0.000572,0.000073,0.000043
07psyrntwf,0.000061,0.000392,0.000042
08rnt87bvt,0.000061,0.000418,0.000044
0biqvjgaqm,0.000061,0.002010,0.000042
0h2plto95e,0.000365,0.000073,0.000043
0rduy8mluy,0.001674,0.000073,0.000049
0rrogcixwm,0.000061,0.000075,0.000234
0sghq6jdqn,0.000062,0.000352,0.000042
100daysofcode,0.000063,0.000074,0.017628
100daysofmlcode,0.000061,0.000797,0.000043


In [9]:
word_weights_df.sort_values(by='Topic_0',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2
learn,0.030244,0.008100,0.000049
google,0.013545,0.000133,0.000197
free,0.012395,0.000143,0.000423
computer,0.011908,0.000076,0.000043
job,0.011870,0.000082,0.002350
science,0.010851,0.000077,0.000045
want,0.010233,0.000091,0.000043
web,0.010069,0.000074,0.004110
today,0.009659,0.000074,0.007410
build,0.008008,0.002793,0.000052


In [10]:
#find out the dominant topic
lda_udacity_output = lda_udacity_corpus.transform(bow_udacity_corpus)
doc_names = ["Doc_" + str(i) for i in range(len(normalized_udacity))]
topic_names = ["Topic_" + str(i) for i in range(no_topics)]
df_document_topic = pd.DataFrame(np.round(lda_udacity_output, 4), columns=topic_names, index=doc_names)
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
df_document_topic[0:4]


,Topic_0,Topic_1,Topic_2,dominant_topic
Doc_0,0.1581,0.6839,0.1580,1
Doc_1,0.9237,0.0384,0.0380,0
Doc_2,0.1462,0.8259,0.0279,1
Doc_3,0.0280,0.0278,0.9442,2


In [11]:
pyLDAvis.enable_notebook()
visualization_panel = pyLDAvis.sklearn.prepare(lda_udacity_corpus, bow_udacity_corpus, bow_vectorizer_udacity, mds='tsne')
visualization_panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2     -372.142456 -195.243271       1        1  44.251853
0     -437.769562 -443.703552       2        1  30.176354
1     -620.127258 -262.638367       3        1  25.571793, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
1360  Default   73.000000        notebook   73.000000  30.0000  30.0000
1310  Default  108.000000      nanodegree  108.000000  29.0000  29.0000
1148  Default  106.000000           learn  106.000000  28.0000  28.0000
489   Default   86.000000            data   86.000000  27.0000  27.0000
496   Default   84.000000             day   84.000000  26.0000  26.0000
932   Default   46.000000            idea   46.000000  25.0000  25.0000
1149  Default   78.000000        learning   78.000000  24.0000  24.0000
8     Default   74.000000   100daysofcode   74.000000  23.0000  23.0000
2050  Default   42.000000           video   42.000000  22.0000  22.0000
1514  Default   71.000000         program   71.000000  21.0000  21.0000
825   Default   41.000000        graffiti   41.000000  20.0000  20.0000
1176  Default   40.000000            live   40.000000  19.0000  19.0000
1948  Default   40.000000             top   40.000000  18.0000  18.0000
1783  Default   36.000000           sound   36.000000  17.0000  17.0000
1519  Default   36.000000  projectjupyter   36.000000  16.0000  16.0000
176   Default   36.000000      annotation   36.000000  15.0000  15.0000
267   Default   35.000000         betatim   35.000000  14.0000  14.0000
1336  Default   73.000000             new   73.000000  13.0000  13.0000
1204  Default   57.000000         machine   57.000000  12.0000  12.0000
814   Default   40.000000          google   40.000000  11.0000  11.0000
415   Default   34.000000        computer   34.000000  10.0000  10.0000
1218  Default   29.000000       marketing   29.000000   9.0000   9.0000
753   Default   37.000000            free   37.000000   8.0000   8.0000
1691  Default   31.000000         science   31.000000   7.0000   7.0000
2081  Default   29.000000            want   29.000000   6.0000   6.0000
158   Default   43.000000          amazon   43.000000   5.0000   5.0000
827   Default   55.000000           great   55.000000   4.0000   4.0000
2137  Default   41.000000            work   41.000000   3.0000   3.0000
474   Default   22.000000        customer   22.000000   2.0000   2.0000
1051  Default   44.000000             job   44.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
176    Topic3   35.761553      annotation   36.118096   1.3538  -4.2201
267    Topic3   34.878984         betatim   35.235557   1.3535  -4.2451
825    Topic3   41.157584        graffiti   41.642568   1.3520  -4.0795
1218   Topic3   29.588836       marketing   29.945616   1.3517  -4.4095
474    Topic3   22.642349        customer   22.998746   1.3481  -4.6771
1388   Topic3   21.416779          online   21.775048   1.3471  -4.7328
1508   Topic3   17.885262         product   18.244552   1.3438  -4.9130
599    Topic3   17.307664             edx   17.664266   1.3433  -4.9458
1464   Topic3   16.473262        platform   16.828624   1.3423  -4.9952
425    Topic3   14.938280         content   15.323504   1.3382  -5.0930
1142   Topic3   13.564900             lay   13.920574   1.3378  -5.1895
1440   Topic3   11.907047         percent   12.262449   1.3343  -5.3198
1393   Topic3   11.907330       operation   12.262830   1.3343  -5.3198
1633   Topic3   11.907342     restructure   12.263047   1.3342  -5.3198
2138   Topic3   11.906928       workforce   12.262877   1.3342  -5.3198
1988   Topic3   12.848664           udemy   13.248036   1.3331  -5.2437
1760   Topic3   10.181621           sneak   10.544175   1.3287  -5.4764
1103   Topic3    9.837345          kotlin   10.194147   1.3281  -5.

**** Evaluation Based on Log-Likelihood, Perplexity and Coherence Scores ****

In [36]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [37]:
udacity_corpus_tokenized = [tokenize_text(normalized_udacity[doc_id]) for doc_id in range(len(normalized_udacity))]
udacity_dictionary = Dictionary(udacity_corpus_tokenized)
udacity_corpus_bow = [udacity_dictionary.doc2bow(doc) for doc in udacity_corpus_tokenized]

In [38]:
cm = CoherenceModel(topics=topic_words, 
                    corpus = udacity_corpus_bow , 
                    dictionary = udacity_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(),4))  # get coherence value
print("Coherence score by topic: ", np.round(cm.get_coherence_per_topic(),4)) #coherence score by topic

Coherence score for the model:  -4.3187
Coherence score by topic:  [-3.038  -6.3236 -4.2539 -3.6592]


In [39]:
print("Log-Likelihood (higher values are better): ", lda_udacity_corpus.score(bow_udacity_corpus))
print("Perplexity (lower values are better): ", lda_udacity_corpus.perplexity(bow_udacity_corpus))

Log-Likelihood (higher values are better):  -64993.95479878974
Perplexity (lower values are better):  925.8235226912636


In [40]:
def select_highest_loglikelihood(lowest, highest):
    candidate = []
    for i in range(lowest,highest+1):
        candidate.append(i)
    no_keywords = 10
    res = []
    for i in range(lowest,highest+1):
        no_topics = i    
        lda_udacity_corpus = LatentDirichletAllocation(n_components=no_topics, max_iter=100,random_state = 42).fit(bow_udacity_corpus)
        res.append(lda_udacity_corpus.score(bow_udacity_corpus))
    idx = res.index(max(res))
    return candidate[idx]
def select_lowest_perplexity(lowest, highest):
    candidate = []
    for i in range(lowest,highest+1):
        candidate.append(i)
    no_keywords = 10
    res = []
    for i in range(lowest,highest+1):
        no_topics = i    
        lda_udacity_corpus = LatentDirichletAllocation(n_components=no_topics, max_iter=100,random_state = 42).fit(bow_udacity_corpus)
        res.append(lda_udacity_corpus.perplexity(bow_udacity_corpus))
    idx = res.index(min(res))
    return candidate[idx]
def select_highest_coherence_score(lowest, highest):
    candidate = []
    for i in range(lowest,highest+1):
        candidate.append(i)
    no_keywords = 10
    res = []
    for i in range(lowest,highest+1):
        no_topics = i    
        lda_udacity_corpus = LatentDirichletAllocation(n_components=no_topics, max_iter=100,random_state = 42).fit(bow_udacity_corpus)
        topic_words = get_topic_words(vectorizer = bow_vectorizer_udacity, 
                              lda_model = lda_udacity_corpus, 
                              n_words = no_keywords)
        cm = CoherenceModel(topics=topic_words, corpus = udacity_corpus_bow , dictionary = udacity_dictionary, coherence='u_mass')
        res.append(np.round(cm.get_coherence(),4))
    idx = res.index(max(res))
    return candidate[idx]

In [41]:
select_highest_loglikelihood(2,5)

C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  Deprecat

5

In [42]:
select_lowest_perplexity(2,5)

C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  Deprecat

5

In [43]:
select_highest_coherence_score(2, 5)

C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  Deprecat

3